<a href="https://colab.research.google.com/github/NiloufarYousefi/-Final-project/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")
!pip install datasets

import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk
nltk.download('punkt_tab')
# Download the stopwords data:
nltk.download('stopwords')  # This line is added to download the stopwords dataset

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')  # This line is added to mount Google Drive

# Load and preprocess dataset
df = pd.read_csv('/content/drive/MyDrive/hate_offensive_data.csv')
del df['Unnamed: 0']

# Label update: "flag" for offensive (hateful) and "unflag" for safe
df['label'] = np.where(df['class'] == 2, 'unflag', 'flag')

# Class "unflag" - Appropriate (Safe)
# Class "flag" - Inappropriate (Hateful or offensive)

df.head()

dataset = load_dataset("imdb")
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text.lower())
    tokens = word_tokenize(text)
    return ' '.join([word for word in tokens if word not in stopwords.words('english')])

dataset = dataset.map(lambda x: {'text': preprocess_text(x['text'])})

# Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
encodings = tokenizer(list(dataset['train']['text']), truncation=True, padding=True, max_length=512)

# Model setup
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

def train():
    model.train()
    for batch in DataLoader(encodings, batch_size=8, shuffle=True):
        optimizer.zero_grad()
        inputs = {key: torch.tensor(val) for key, val in batch.items()}
        outputs = model(**inputs)
        outputs.loss.backward()
        optimizer.step()

def evaluate():
    model.eval()
    with torch.no_grad():
        for batch in DataLoader(encodings, batch_size=8):
            inputs = {key: torch.tensor(val) for key, val in batch.items()}
            predictions = torch.argmax(model(**inputs).logits, dim=1)
            # Print the predicted labels as 'flag' or 'unflag'
            print(['flag' if pred == 1 else 'unflag' for pred in predictions])

train()
evaluate()


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]